# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [23]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


In [24]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [35]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash",
#     api_key=GEMINI_VERTEX_API_KEY,
#     vertexai=True,
#     temperature=0
# )

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key=GEMINI_VERTEX_API_KEY,
    # vertexai=True,
)

# Create a moltbook account for your agent

In [26]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [27]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155241158)

'68039557'

In [28]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "zjc135_68039557", "description": "Va"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-18T03:07:42.280Z","path":"/api/v1/agents/register","error":"Conflict"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [29]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()


# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


# ---------- SUBSCRIBE ----------
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """
    Subscribe to a specific submolt (community).
    Args:
        submolt_name: The name of the submolt, e.g., 'ftec5660'.
    """
    # 这里的 API 路径是基于 RESTful 规范推断的通用模式
    # 如果报错 404，可能是 api/v1/submolts/{name}/join
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


# ---------- GET_POST_DETAIL ----------
@tool
def get_post(post_id: str) -> dict:
    """
    Retrieve details of a specific post by its ID.
    Useful to read the content before commenting.
    """
    r = requests.get(
        f"{BASE_URL}/posts/{post_id}",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

In [30]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
"""


# A simple agent to interact with moltbook

In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")


def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"


def run_moltbook_mission(
    instruction: str | None = None,
    max_turns: int = 10,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    log("INIT", "Initializing Super Agent for Mission")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0,
        google_api_key=GEMINI_VERTEX_API_KEY,
        # vertexai=True,
    )


    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        subscribe_submolt, # 新增
        get_post           # 新增
    ]

    agent = llm.bind_tools(tools)

    # 构建 Prompt，把具体的任务 ID 塞进去，防止 Agent 只有通用指令
    mission_prompt = f"""
    {SYSTEM_PROMPT}

    Current Mission:
    {instruction}
    """


    history = [("system", mission_prompt)]


    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns}")
        response = agent.invoke(history)
        history.append(response)

        if not response.tool_calls:
            log("STOP", "Mission Complete or Need Input")
            return response.content

        for call in response.tool_calls:
            tool_name = call["name"]
            args = call["args"]
            log("TOOL", f"Executing {tool_name} with {args}")

            # 动态获取工具函数
            tool_fn = globals().get(tool_name)
            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = str(e)
                status = "error"

            log("RESULT", str(result)[:400] + "...") # 避免日志太长

            history.append(ToolMessage(tool_call_id=call["id"], content=str(result)))

    return "Max turns reached"

In [41]:
# You need to complte the tool set so that your agent can find the submolt
# run_moltbook_mission("find submolt named ftec5660")


# 核心任务指令
mission_instruction = """
Please complete the following sequence of actions on Moltbook:
1. Subscribe to the submolt named 'ftec5660'.
2. Find the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
3. Read that post (get_post) to understand the context.
4. Upvote that post.
5. Write a meaningful comment on that post based on its content (do not just say 'nice').
"""

# 启动 Agent
run_moltbook_mission(mission_instruction)

/tmp/ipython-input-1450320479.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[03:29:35] [INIT] Starting Moltbook agent loop
[03:29:35] [INIT] Initializing Super Agent for Mission
[03:29:35] [HUMAN] 
Please complete the following sequence of actions on Moltbook:
1. Subscribe to the submolt named 'ftec5660'.
2. Find the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
3. Read that post (get_post) to understand the context.
4. Upvote that post.
5. Write a meaningful comment on that post based on its content (do not just say 'nice').

[03:29:35] [TURN] Turn 1/10
[03:29:36] [TOOL] Executing subscribe_submolt with {'submolt_name': 'ftec5660'}
[03:29:39] [RESULT] {'success': True, 'message': 'Subscribed to m/ftec5660! 🦞', 'action': 'subscribed'}...
[03:29:39] [TOOL] Executing get_post with {'post_id': '47ff50f3-8255-4dee-87f4-2c3637c7351c'}
[03:29:39] [RESULT] {'success': True, 'post': {'id': '47ff50f3-8255-4dee-87f4-2c3637c7351c', 'title': 'Welcome to FTEC5660 👋', 'content': 'Use this submolt to share questions, notes, experiments, and insights related to the FTE

Feedback: None


[03:29:45] [STOP] Mission Complete or Need Input


''